In [1]:
# Import MNIST dataset from torchvision
import torch
import torchvision
from torch.utils.data import random_split, DataLoader, TensorDataset
import torch.nn as nn

# Load Data
mnist_dataset = torchvision.datasets.MNIST(root = ".", download = True)

train_size = int(0.9 * len(mnist_dataset))
val_size = len(mnist_dataset) - train_size

# Train / Val Split
train_dataset, val_dataset = random_split(mnist_dataset, [train_size, val_size])

train_x_data = train_dataset.dataset.data[train_dataset.indices]
train_y_data = train_dataset.dataset.targets[train_dataset.indices]
val_x_data   = val_dataset.dataset.data[val_dataset.indices]
val_y_data   = val_dataset.dataset.targets[val_dataset.indices]

# Comb TensorDataset
train_tensor_dataset = TensorDataset(train_x_data, train_y_data)
val_tensor_dataset = TensorDataset(val_x_data, val_y_data)

# DataLoader
train_loader = DataLoader(train_tensor_dataset, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(val_tensor_dataset, batch_size=128, shuffle=False, num_workers=0)

100%|██████████| 9.91M/9.91M [00:00<00:00, 56.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.61MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 12.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.10MB/s]


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
  # Your Model is Defined Here
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Conv2d(1, 6, kernel_size=5, padding=2),
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Conv2d(6, 16, kernel_size=5),
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(16 * 5 * 5, 120),
        nn.Linear(120, 84),
        nn.Linear(84,10)
    )

  def forward(self, x):
    return self.net(x)

model = LeNet()

In [3]:
from tqdm import tqdm

max_epochs = 40
learning_rate = 5 * 1e-5
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in tqdm(range(max_epochs)):
    model.train()
    total = 0
    correct = 0
    cur_loss = 0.0

    for train_x, train_y in train_loader:
        train_x = train_x.float().to(device)       # [batch, 28, 28]
        train_y = train_y.to(device)               # [batch]
        if train_x.dim() == 3:
            train_x = train_x.unsqueeze(1)         # [batch, 1, 28, 28]

        predict_y = model(train_x)                 # [batch, 10]
        loss = criterion(predict_y, train_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cur_loss += loss.item() * train_x.size(0)  # total loss
        preds = predict_y.argmax(dim=1)            # [batch]
        correct += (preds == train_y).sum().item()
        total += train_x.size(0)

    avg_loss = cur_loss / total
    acc = correct / total

    val_total = 0
    val_correct = 0
    for val_x, val_y in val_loader:
      val_x = val_x.float().to(device)
      val_y = val_y.to(device)
      if val_x.dim() == 3:
          val_x = val_x.unsqueeze(1)         # [batch, 1, 28, 28]
      predict_val_y = model(val_x)                 # [batch, 10]

      preds = predict_val_y.argmax(dim=1)            # [batch]
      val_correct += (preds == val_y).sum().item()
      val_total += val_x.size(0)

    val_acc = val_correct / val_total
    print(f"Epoch {epoch}: Loss={avg_loss:.4f}, Acc={acc:.4f}, Val_Acc={val_acc:.4f}")


  2%|▎         | 1/40 [00:19<12:24, 19.09s/it]

Epoch 0: Loss=2.2683, Acc=0.1936, Val_Acc=0.3515


  5%|▌         | 2/40 [00:39<12:24, 19.59s/it]

Epoch 1: Loss=1.6770, Acc=0.5093, Val_Acc=0.6595


  8%|▊         | 3/40 [00:58<12:03, 19.54s/it]

...

Epoch 37: Loss=0.1332, Acc=0.9598, Val_Acc=0.9555


 98%|█████████▊| 39/40 [13:23<00:21, 21.10s/it]

Epoch 38: Loss=0.1302, Acc=0.9606, Val_Acc=0.9570


100%|██████████| 40/40 [13:44<00:00, 20.61s/it]

Epoch 39: Loss=0.1275, Acc=0.9609, Val_Acc=0.9577


In [19]:
# Input PIL image to test
from PIL import Image
from google.colab import files
from torchvision import transforms
import matplotlib.pyplot as plt

uploaded = files.upload()
filename = next(iter(uploaded))
img = Image.open(filename)
#img = Image.open('test_1.jpg')
#img = img.resize((28, 28))
#img = img.convert('L')

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(),
    transforms.Lambda(lambda x: transforms.functional.invert(x)),
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
])
img = transform(img)
img = img.float().to(device)
img = img.unsqueeze(1)

propabilities = model(img)
result = propabilities.argmax()
print(f"result={result}, propabilities={propabilities}")

Saving test_8.jpg to test_8 (1).jpg
result=7, propabilities=tensor([[-4.8400,  4.3792,  2.1376,  1.7936, -3.9108,  4.9050, -7.0795,  5.0249,
         -6.0566, -1.3205]], grad_fn=<AddmmBackward0>)


In [8]:
print(img)

tensor([[[[0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
           0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
           0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
           0.9922, 0.9922, 0.9922, 0.9922],
          [0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
           0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
           0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922, 0.9922,
           0.9922, 0.9922, 0.9922, 0.9922],
          [0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961,
           0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961,
           0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961,
           0.9961, 0.9961, 0.9961, 0.9961],
          [0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961,
           0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961, 0.9961,
           0.9961, 0.9961, 0.9961, 0.9961, 